This (still pretty simple) example counts words in a file and keeps track of some metrics about the data as it's processed.  The input will again be Much Ado About Nothing; the output will be "less_simple_counts"

In [1]:
import logging
logging.getLogger().setLevel(logging.ERROR)
logging.basicConfig()

import re
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import StandardOptions

from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

input_file = "muchAdo.txt"
output_file = "less_simple_counts"

This time, instead of just defining a function, we'll make a class that inherits from ParDo so we can keep track of our metrics 

In [2]:
class WordExtractingDoFn(beam.DoFn):
    """Parse each line of input text into words."""

    def __init__(self):
        super(WordExtractingDoFn, self).__init__()
        self.word_counter = Metrics.counter(self.__class__, 'num_words')
        self.word_lengths_dist = Metrics.distribution(self.__class__, 'word_len_dist')
        
    def process(self, line):
        text_line = line.strip()
        words = re.findall(r'[A-Za-z\']+', text_line)
        for word in words:
            self.word_counter.inc()
            self.word_lengths_dist.update(len(word))
        return words


Next, we'll set up the pipeline and the output.  This is all exactly the same as in the previous file, except the name of the function was replaced with the name of the class.

In [3]:
options = PipelineOptions()
options.view_as(StandardOptions).runner = 'DirectRunner'

p = beam.Pipeline(options=options)

lines = p | 'read' >> ReadFromText(input_file)

counts = (lines
          | "split" >> (beam.ParDo(WordExtractingDoFn()).with_output_types(unicode))
          | "pair_with_1" >> beam.Map(lambda x: (x, 1))
          | "group" >> beam.GroupByKey()
          | "count" >> beam.Map(lambda(x, ones): (x, sum(ones)))
         )

output = counts | 'format' >> beam.Map(lambda (word, c): '%s: %s' % (word, c))
output | 'write' >> WriteToText(output_file)

<PCollection[write/Write/WriteImpl/FinalizeWrite.None] at 0x7ff7c0ca9050>

Again, once the pipeline is set up we want to run it 

In [4]:
result = p.run()

Finally, we want to see the results of our metrics, so we wait until the pipeline is finished and view the results:

In [5]:
result.wait_until_finish()

word_lengths_filter = MetricsFilter().with_name('word_len_dist')
query_result = result.metrics().query(word_lengths_filter)
if query_result['distributions']:
  word_lengths_dist = query_result['distributions'][0]
  print 'Average word length: ' + str(word_lengths_dist.committed.mean)
    
num_words_filer = MetricsFilter().with_name('num_words')
query_result = result.metrics().query(num_words_filer)
if query_result['counters']:
  total_words = query_result['counters'][0]
  print 'Number of total words: ' + str(total_words.committed)

Average word length: 4.08304042179
Number of total words: 22760


Now we can see the results (which shouldn't be any different from the previous results)

In [ ]:
! diff "simple_counts-00000-of-00001" "less_simple_counts-00000-of-00001"

In [ ]:
! more "less_simple_counts-00000-of-00001"

sunburnt: 1
pardon: 4
needful: 1
foul: 8
four: 2
hath: 67
protest: 4
sleep: 2
friend's: 1
hanging: 1
appetite: 1
evermore: 1
saved: 1
yonder: 1
conjure: 1
muzzle: 1
vile: 2
crept: 1
'Shall: 1
Watch: 5
endings: 1
neighbours: 2
MUCH: 18
--More--(0%)